## Observations and Insights 

In [85]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
mouse_study_df = pd.merge(mouse_metadata,study_results)
# Display the data table for preview
mouse_study_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [86]:
# Checking the number of mice.
mouse_count = len(mouse_study_df['Mouse ID'].value_counts())
mouse_count

249

In [87]:
#first drop all duplicate rows of data held in the dataset
duplicates = mouse_study_df.duplicated()
cleaned_mouse_study_df = mouse_study_df.drop_duplicates()

#next drop duplicates within the Mouse ID category that have the same Timpoint.  Keep the latest entry.
cleaned_mouse_study_df = cleaned_mouse_study_df.drop_duplicates(
                        subset=['Mouse ID', 'Timepoint'],
                        keep = 'last').reset_index(drop = True)
cleaned_mouse_study_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1883,z969,Naftisol,Male,9,30,25,63.145652,2
1884,z969,Naftisol,Male,9,30,30,65.841013,3
1885,z969,Naftisol,Male,9,30,35,69.176246,4
1886,z969,Naftisol,Male,9,30,40,70.314904,4


In [57]:
# Optional: Get all the data for the duplicate mouse ID. 



In [5]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.


In [88]:
# Checking the number of mice in the clean DataFrame.
cleaned_mouse_count = len(cleaned_mouse_study_df['Mouse ID'].value_counts())
cleaned_mouse_count

249

## Summary Statistics

In [113]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen 
cleaned_mouse_study_df.head()
regimen_table = cleaned_mouse_study_df.groupby('Drug Regimen').sum()
tumor_mean = cleaned_mouse_study_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].mean()
tumor_mean
tumor_meadian = cleaned_mouse_study_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].median()
tumor_meadian
tumor_variance = cleaned_mouse_study_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].var()
tumor_variance
tumor_stddev = cleaned_mouse_study_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].std()
tumor_stddev
tumor_SEM = cleaned_mouse_study_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].sem()
tumor_SEM
tumor_summary_df = pd.merge(tumor_mean, tumor_meadian, on='Drug Regimen')
tumor_summary_df.rename(columns={"Tumor Volume (mm3)_x":"Tumor Volume Mean (mm3)",'Tumor Volume (mm3)_y':'Tumor Volume Median (mm3)'}, inplace = True)
tumor_summary_df
#regimen_table = pd.DataFrame(regimen_table)
#regimen_table
#tumor_mean = np.mean(regimen_table.iloc[:,3])
#tumor_mean = regimen_table['Tumor Volume (mm3)']
#tumor_mean
#cleaned_mouse_study_df['Tumor Volume (mm3)'].describe()
#tumor_meadian = cleaned_mouse_study_df['Tumor Volume (mm3)'].median()
#tumor_stddev = cleaned_mouse_study_df['Tumor Volume (mm3)'].std()
#tumor_variance = cleaned_mouse_study_df['Tumor Volume (mm3)'].
#tumor_sem = cleaned_mouse_study_df['Tumor Volume (mm3)'].SEM()

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.



,Tumor Volume Mean (mm3),Tumor Volume Median (mm3)
Drug Regimen,,
Capomulin,40.675741,41.557809
Ceftamin,52.591172,51.776157
Infubinol,52.884795,51.820584
Ketapril,55.235638,53.698743
Naftisol,54.331565,52.509285
Placebo,54.033581,52.288934
Propriva,52.382993,50.783528
Ramicane,40.216745,40.673236
Stelasyn,54.233149,52.431737


In [83]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using Pandas.



In [10]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
